In [ ]:
%pylab inline
%matplotlib inline

import importlib
import glob
import simulatorutils
import pandas as pd
importlib.reload(simulatorutils)
from simulatorutils import MultiBandAuctionState, VCGState
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from matplotlib.ticker import FuncFormatter
import operator
import subprocess
import matplotlib.colors as colors
from statsmodels.distributions.empirical_distribution import ECDF

import tempfile
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

from collections import defaultdict
from matplotlib import rc
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from scipy.stats import wilcoxon

In [ ]:
FIGDIR = '/ubc/cs/research/arrow/satfc/satfc-scripts/simulator/newish/newest/figures'
COST = 'Total Cost'
EFFICIENCY = 'Total Value Loss'

DEFAULT_MARKERSIZE = 120
MARKERSIZE = DEFAULT_MARKERSIZE
sns.set(font_scale=4, style='whitegrid')
FIGSIZE = (18,12)

RASTERIZE_POINTS = False

normalized = True
# op = operator.sub 
op = operator.truediv
means_only = False

In [ ]:
def limits_from_frame(df, reference_type='FCC'):
    df = df.reset_index().copy().set_index(['auction', 'model', 'UHF_Only'])
    reference_type = df.query(f'type == "{reference_type}"')
#     display(reference_type)
    top = (df[COST] / reference_type[COST]).max()
    bottom = (df[COST] / reference_type[COST]).min()
    left = (df[EFFICIENCY] / reference_type[EFFICIENCY]).min()
    right = (df[EFFICIENCY] / reference_type[EFFICIENCY]).max()
    print((left, right, top, bottom))
    return (left, right, bottom, top)


In [ ]:
def special_save_fig(fig, file_name, fmt=None, dpi=300, tight=True):
    """Save a Matplotlib figure as EPS/PNG/PDF to the given path and trim it.
    """
    if not fmt:
        fmt = file_name.strip().split('.')[-1]

    if fmt not in ['eps', 'png', 'pdf']:
        raise ValueError('unsupported format: %s' % (fmt,))

    extension = '.%s' % (fmt,)
    if not file_name.endswith(extension):
        file_name += extension

    file_name = os.path.abspath(file_name)
    
    with tempfile.NamedTemporaryFile() as tmp_file:
        tmp_name = tmp_file.name + extension

    # save figure
    if tight:
        fig.savefig(tmp_name, dpi=dpi, bbox_inches='tight')
    else:
        fig.savefig(tmp_name, dpi=dpi)

    #trim it
    if fmt == 'eps':
        subprocess.call('epstool --bbox --copy %s %s' %
                        (tmp_name, file_name), shell=True)
    elif fmt == 'png':
        subprocess.call('convert %s -trim %s' %
                        (tmp_name, file_name), shell=True)
    elif fmt == 'pdf':
        subprocess.call('pdfcrop %s %s' % (tmp_name, file_name), shell=True)

In [ ]:
def name_to_extra(d, name, include_label=True):
    retval = dict(d.get(name, {}))
    retval['label'] = retval.get('label', name)
    if not include_label:
        del retval['label']
    return retval

In [ ]:
def plotting_code(FRAMES, normalized, d, op=operator.truediv, means=True, means_only=False, color_is_stage=False):
    cNorm  = colors.Normalize(vmin=1, vmax=9)
    cmap = plt.cm.get_cmap('tab10', 9)
    base_kwargs = dict()
    if color_is_stage:
        base_kwargs['cmap'] = cmap
        base_kwargs['norm'] = cNorm
    
    baseline = FRAMES[0]
    fig = plt.figure(figsize=FIGSIZE)
    for ii, frame in enumerate(FRAMES):
        name = frame['type'].unique()[0]
        if normalized:         
            if not means_only and ii > 0:
                scatter = plt.scatter(x=op(frame[EFFICIENCY], baseline[EFFICIENCY]), y=op(frame[COST],baseline[COST]), sizes=[MARKERSIZE]*len(frame), **name_to_extra(d, name), alpha=0.6, rasterized=RASTERIZE_POINTS, zorder=10)
                scatter.set_clip_on(False)
            if means or ii == 0:
                mean_eff = op(frame[EFFICIENCY].mean(), baseline[EFFICIENCY].mean())
                mean_cost = op(frame[COST].mean(), baseline[COST].mean())
                print(f"Mean value loss of {name} is {mean_eff} of reference. Mean cost is {mean_cost} of reference")
                extra_args = name_to_extra(d, name, include_label=means_only or ii==0)
                if 'marker' in extra_args:
                    del extra_args['marker']
                if 'facecolor' in extra_args:
                    del extra_args['facecolor']
                scatter = plt.scatter(x=[mean_eff], y=[mean_cost], sizes=[MARKERSIZE*8], marker='*' if ii > 0 else 'd',rasterized=RASTERIZE_POINTS, zorder=10, **extra_args)
                scatter.set_clip_on(False)
        else:
            if means:
                extra_args = name_to_extra(d, name, include_label=means_only)
                if 'marker' in extra_args:
                    del extra_args['marker']
                if 'facecolor' in extra_args:
                    del extra_args['facecolor']
                scatter = plt.scatter(x=frame[EFFICIENCY].mean(), y=frame[COST].mean(), sizes=[MARKERSIZE*5], marker='*',rasterized=RASTERIZE_POINTS, zorder=10, **extra_args)
                scatter.set_clip_on(False)
            if not means_only:
                kwargs = dict(base_kwargs)

                tmp = name_to_extra(d, name)                
                    
                scatter = plt.scatter(x=frame[EFFICIENCY], y=frame[COST], sizes=[MARKERSIZE]*len(frame), rasterized=RASTERIZE_POINTS, **tmp, zorder=10, **kwargs)
                scatter.set_clip_on(False)

    return fig

In [ ]:
def format_normalized(df, baseline, op=operator.truediv, set_limits=True):
    '''Limits is (left, right, top, bottom) tuple or None'''
    op_to_rep = {operator.truediv: '/', operator.sub: '(Billions) -'}
    ax = plt.gca()
    baseline_name = baseline['type'].unique()[0]
    if op == operator.truediv:
        plt.xlabel(f'Normalized Value Loss')
        plt.ylabel(f'Normalized Cost')
#         plt.xlabel(f'Value Loss {op_to_rep[op]} {baseline_name} Value Loss')
#         plt.ylabel(f'Cost {op_to_rep[op]} {baseline_name} Cost')
    else:
        plt.xlabel(f'Value Loss {op_to_rep[op]} {baseline_name} Value Loss')
        plt.ylabel(f'Cost {op_to_rep[op]} FCC Cost')
    
    if set_limits:
        FUDGE = 0.005 if op is operator.truediv else 0.1e9
        plt.xlim(left=op(df[EFFICIENCY], baseline[EFFICIENCY]).min() - FUDGE, right=max(FUDGE, op(df[EFFICIENCY], baseline[EFFICIENCY]).max() + FUDGE))
        plt.ylim(bottom=op(df[COST], baseline[COST]).min() - FUDGE, top=max(op(df[COST], baseline[COST]).max() + FUDGE, FUDGE))
    
    # Add dotted lines
    plt.vlines(1, ax.get_ylim()[0], ax.get_ylim()[1], alpha=0.5, clip_on=False, linestyles='--')
    plt.hlines(1, ax.get_xlim()[0], ax.get_xlim()[1], alpha=0.5, clip_on=False,linestyles='--')
    
    if op == operator.sub:
        ax.xaxis.set_major_formatter(FuncFormatter(lambda x,y : str(x/1e9)))
        ax.yaxis.set_major_formatter(FuncFormatter(lambda x,y : str(x/1e9)))   

#     plt.axis('scaled')
    
def format_unnormalized(set_limits=True):
    ax = plt.gca()
    plt.xlabel('Value Loss (Billions)')
    plt.ylabel('Cost (Billions)')
    plt.ylim(bottom=0)
    plt.xlim(left=0)
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x,y : str(x/1e9)))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x,y : str(x/1e9)))   
    
def format_figure(df, baseline, normalized, d, op=operator.truediv, limits=None, fixed_legend=False):
    if limits is not None:
        if len(limits) != 4:
            raise ValueError("Expected left right top bottom tuple")
        plt.xlim(left=limits[0], right=limits[1])
        plt.ylim(bottom=limits[2], top=limits[3])
    
    if normalized:
        format_normalized(df, baseline, op, set_limits=limits is None)
    else:
        format_unnormalized(set_limits=limits is None)
    
    if limits is None:
        print(plt.gca().get_xlim(), plt.gca().get_ylim())
        
#     plt.legend(loc='best')
    axis = plt.gca()
    handles, labels = axis.get_legend_handles_labels()
    
#     for h, l in zip(handles, labels):
#         handles = mlines.Line2D([], [], color='black', marker='*', linestyle='None',
#                           markersize=10, label='Blue stars')

    
#         c = 'red'
#     #     c = None
#     #     for k in d.keys():
#     #         if d[k]['label'] == labels[0]:
#     #             c = d[k]['color']
#     #             break
#     #     if c is None:
#     #         raise ValueError()

#         handles[0] = mpatches.Patch(color=c)
    
    
    # Sort according to efficiency
    if len(labels) > 1 and not fixed_legend:
        mean_eff = df.groupby(level=0)[EFFICIENCY].mean().sort_values().index.values
        order = []
        for l in mean_eff:
            try:
                order.append(labels.index(l))
            except Exception as e:
                order.append(labels.index(d[l]['label']))
        order = np.array(order)
    else:
#         order = np.array(list(range(len(labels))))
        order = np.argsort(labels)
        print(order)
    
#     if normalized:
#         labels[0] = '$$\textbf{lab}$$'

    
    legend = axis.legend(handles=list(np.array(handles)[order]),labels=list(np.array(labels)[order]), loc='best') 

#     legend = axis.legend(handles=list(np.array(handles)[order]),labels=list(np.array(labels)[order]), loc='best') 
    
def save_fig(name, normalized, op, means_only=False, uhf_only=False, fig=None, model=None):
    op_to_rep = {operator.truediv: 'relative', operator.sub: 'absolute'}
    fname = name
    if normalized:
        fname += f'_{op_to_rep[op]}'
    if means_only:
        fname += '_means_only'
    if model:
        fname += '_' + model
    if uhf_only:
        fname += '_UHF_ONLY'
    special_save_fig(fig, os.path.join(FIGDIR, f'{fname}.pdf'))

In [ ]:
def parse_experiment(folders, skip_failures=False, delete_failures=False, count_rounds=False, end_only=True, extra=None, specific_end_stage=None, return_states=False, limit=None):
    records = []
    states = []
    if count_rounds and end_only:
        raise ValueError("Count rounds only works if end only is set to false")
    for auction_folder in tqdm(folders):
        try:
            basename = os.path.basename(auction_folder)
            seed = int(basename.split('_')[-1])
            auction_type = auction_folder.split('/')[-2]
            is_vcg = 'VCG' in auction_type
            if is_vcg:
                state = VCGState(auction_folder, all_stations=STATIONS, allow_intermediate=True)
            else:
                state = MultiBandAuctionState(auction_folder, end_only=end_only, specific_end_stage=specific_end_stage)
            cost = state.total_cost()
            value_loss = state.total_value_loss()
            record = {
                'auction': seed,
                'type': auction_type,
                'Total Cost': cost,
                'Total Value Loss': value_loss,
                'UHF_Only': 'UHF_ONLY' in auction_folder,
                'model': 'pop' if '_POP' in auction_folder else 'ulrich',
                'walltime': state.walltime(),
                'cputime': state.cputime(),
                'Final Stage': 1 if is_vcg else state.ending_stage()
            }
            if count_rounds:
                record['n_rounds'] = state.n_rounds()
            if extra is not None:
                for k,v in extra.items():
                    record[k] = v(state)
            records.append(record)
            if return_states:
                states.append(state)
        
        except Exception as e:
            if isinstance(e, KeyboardInterrupt): # Want to be able to use stop button
                raise
            if delete_failures:
                print(f"Deleting {auction_folder}")
                !rm -rf {auction_folder}
            elif skip_failures:
                print(f"Skipping {auction_folder}")
            else:
                raise
                
        if limit is not None and len(records) > limit:
            break
    df = pd.DataFrame.from_records(records)
    df = df.set_index(['type', 'auction'])
    if return_states:
        return df.sort_index(), states
    else:
        return df.sort_index()

In [ ]:
def make_frames(df, types=None, reference_types=None):
    if reference_types is None:
        reference_types = ['FCC', '29']
    if types is None:
        types = df.index.get_level_values(0).unique().values.tolist()
    else:
        types = list(types)
    q = [t for t in types if t in reference_types]
    if len(q) > 0: # Ensure FCC is always first
        types.remove(q[0])
        types = [q[0]] + types
    return [df.reset_index()[df.reset_index()['type'] == x].sort_values('auction').set_index('auction') for x in types if x in df.reset_index()['type'].unique()]

In [ ]:
def make_folders(path, uhf_only=False):
    path += '*/*/*'
    return glob.glob(path)

In [ ]:
def standard_pallet(types):
    d = dict()
    for i, t in enumerate(types):
        d[t] = dict(PALLET[i])
    return d

def standard_analysis(d, name, df, types=None, reference_types=None, means=None, means_only=None, limits=None, stages=None, model=False, save=True, fixed_legend=False):
    if means is None:
        means = True
    if means_only is None:
        means_only = False
    if stages is None:
        stages = False
        
    if df['Final Stage'].nunique() > 1:
        logging.info(f"Multiple stages detected {df['Final Stage'].unique()}")
        if not stages:
            logging.warning("Not showing the difference between stages!")
        if normalized:
            # Most of the time meaningless because you actually care about AMOUNT cleared which somehow isn't in your stupid JSON
            logging.warning("Potentially you are trying a normalized comparison with different amounts of spectrum!")

    FRAMES = make_frames(df, types=types, reference_types=reference_types)
    if df['UHF_Only'].nunique() > 1 and not name.startswith('vhf'):
        raise ValueError("DF mixes UHF only and non-UHF only")
    uhf_only = df['UHF_Only'].all()
    if df['model'].nunique() > 1:
        raise ValueError(f"DF mixes value models! {df['model'].unique()}")
    
    fig = plotting_code(FRAMES, normalized, d, op=op, means_only=means_only, means=means, color_is_stage=stages)
    format_figure(df, FRAMES[0], normalized, d, op=op, limits=limits, fixed_legend=fixed_legend) 
    if save:
#         plt.legend(loc='upper left')
        save_fig(name, normalized, op, means_only=means_only, uhf_only=uhf_only, fig=fig, model=df['model'].unique()[0])
    
def dual_standard_analysis(d, name, df, types=None, reference_types=None, means=None, means_only=None, limits=None, auto_limits=True, save=True, fixed_legend=False):
    if auto_limits and limits is None:
        limits = limits_from_frame(df, reference_type=reference_types[0] if reference_types is not None else types[0] if types is not None else 'FCC')
    for model, gdf in df.groupby('model'):
        print(f"Model {model}")
        uhf_only = gdf.loc[gdf['UHF_Only'] == True]
        not_uhf_only = gdf.loc[gdf['UHF_Only'] == False]
        kwargs = dict(types=types, reference_types=reference_types, means=means, means_only=means_only, limits=limits, model=model, save=save, fixed_legend=fixed_legend)
        if len(uhf_only) > 0:
            print(f"UHF ONLY")
            standard_analysis(d, name, uhf_only, **kwargs)
        if len(not_uhf_only) > 0:
            print(f"UHF+VHF")
            standard_analysis(d, name, not_uhf_only, **kwargs)

In [ ]:
PALLET = [
    {
        'color': 'red',
        'marker': 'o',
        'facecolor': 'none',
        'linewidth': 1
    },
    {
        'color': 'blue',
        'marker': 's',
        'facecolor': 'none',
        'linewidth': 1
    },
    {
        'color': 'black',
        'marker': '^',
        'facecolor': 'none',
        'linewidth': 1
    },
    {
        'color': 'green',
        'marker': 'D',
        'facecolor': 'none',
        'linewidth': 1
    },
    {
        'color': 'purple'
    },
    {
        'color': 'orange'
    },
    {
        'color': 'pink'
    }
]

In [ ]:
scoring_df = parse_experiment(make_folders('/global/scratch/satfc/new_experiments/v6/scoring'), skip_failures=True)

In [ ]:
# Interference here is obviously going to including VHF constraints, even in the UHF-only case. I guess that's OK, since they don't vary it either between channels. 
FCC_SCORING = 'FCC'
INTERFERENCE_SCORING = 'Interference'
POPULATION_SCORING = 'Population'
UNIFORM_SCORING = 'Uniform'

HALF_INTERFERENCE = 'HalfInterference'
HALF_POPULATION = 'HalfPopulation'
# TYPES = [FCC_SCORING, INTERFERENCE_SCORING, POPULATION_SCORING, UNIFORM_SCORING]
TYPES = [FCC_SCORING, HALF_INTERFERENCE, HALF_POPULATION, UNIFORM_SCORING]

d = standard_pallet(TYPES)
d[FCC_SCORING]['label'] = 'Incentive Auction'#r'$\sqrt{Population} \cdot \sqrt{Interference}$'
d[HALF_INTERFERENCE]['label'] = 'Interference'
d[HALF_POPULATION]['label'] = 'Population'
print(f"CPU time for scoring: {scoring_df.loc[TYPES]['cputime'].sum()}")


dual_standard_analysis(d, 'scoring', scoring_df.loc[TYPES], types=TYPES)

In [ ]:
timeout_df = parse_experiment(make_folders('/global/scratch/satfc/new_experiments/v6/no_price_drop_timeouts'), skip_failures=True)

In [ ]:
TYPES = timeout_df.index.levels[0].values
d = standard_pallet(TYPES)
d['FCC']['label'] = 'Incentive Auction'
d['NO_PRICE_DROPS']['label'] = 'No Freezing on Timeouts'
dual_standard_analysis(d, 'timeouts', timeout_df, types=TYPES)

In [ ]:
stage_df = parse_experiment(make_folders('/global/scratch/satfc/new_experiments/v6/multiple_stages'), skip_failures=True)

In [ ]:
logging.info(f"CPU time for stages: {stage_df['cputime'].sum()}")

TWENTY_NINE = '29'
THIRTY_ONE = '31'
THIRTY_TWO = '32'
THIRTY_SIX = '36'
THIRTY_EIGHT = '38'
THIRTY_NINE = '39'
FORTY_ONE = '41'

THIRTY_ONE_TO_THIRTY_SIX = '31_to_36'
TWENTY_NINE_TO_THIRTY_SIX = '29_to_36'

TYPES = [TWENTY_NINE, THIRTY_ONE, THIRTY_TWO, THIRTY_SIX, THIRTY_ONE_TO_THIRTY_SIX, TWENTY_NINE_TO_THIRTY_SIX]

d = standard_pallet(TYPES)
d[TWENTY_NINE]['label'] = '4 Stages'
d[THIRTY_ONE]['label'] = '3 Stages'
d[THIRTY_TWO]['label'] = '2 Stages'
d[THIRTY_SIX]['label'] = '1 Stage'
d[TWENTY_NINE_TO_THIRTY_SIX]['label'] = '2 Stages (126, 84)'
d[THIRTY_ONE_TO_THIRTY_SIX]['label'] = '2 Stages (114, 84)'

no_skipping = stage_df.query(f'type != "{THIRTY_ONE_TO_THIRTY_SIX}" and type != "{TWENTY_NINE_TO_THIRTY_SIX}"')

dual_standard_analysis(d, 'stages_rel36', no_skipping, types=TYPES, reference_types=[THIRTY_SIX], fixed_legend=True)


In [ ]:
(stage_df.loc[TWENTY_NINE].groupby(['model', 'UHF_Only'])[COST].mean() / stage_df.loc[THIRTY_SIX].groupby(['model', 'UHF_Only'])[COST].mean())


In [ ]:
es_df = parse_experiment(make_folders('/global/scratch/satfc/new_experiments/v6/early_stopping'), skip_failures=True)

In [ ]:
compare_to_oracle_df = es_df.loc[['Early_0.67', 'SingleStage_0.67', 'Early_1.00', 'SingleStage_1.00']]
compare_to_oracle_df = compare_to_oracle_df.reset_index()
compare_to_oracle_df['type'] = compare_to_oracle_df['type'].apply(lambda x: x.split('_')[0])
compare_to_oracle_df = compare_to_oracle_df.set_index(['type', 'auction'])
TYPES = ['SingleStage', 'Early']
d = standard_pallet(TYPES)
d['Early']['label'] = 'Early Stopping'
d['SingleStage']['label'] = 'Single Stage Oracle'
dual_standard_analysis(d, f'SSvsES', compare_to_oracle_df, reference_types=['SingleStage'])

In [ ]:
# TODO: Special limits here...
for scale in ['0.33', '0.67', '1.00']:
    TYPES = [f'SingleStage_{scale}', f'Early_{scale}']
    q = es_df.loc[TYPES]
    d = standard_pallet(TYPES)
    d[f'Early_{scale}']['label'] = 'Early Stopping'
    d[f'SingleStage_{scale}']['label'] = 'Single Stage Oracle'
    s = str(float(scale) * 100).strip('0.')
    dual_standard_analysis(d, f'SSvsES_{s}', q, types=TYPES)

In [ ]:
# es_df['Scaling'] = es_df.reset_index()['type'].str.extract('(\d+\.?\d*)').values
# es_df['Historic'] = es_df.reset_index()['type'].str.contains("Historic").values
# es_df['ScalingHistoric'] = es_df['Scaling'].astype(str) + es_df['Historic'].astype(str)

no_ss = es_df.loc[['Early_0.33', 'Early_0.67', 'Early_1.00', 'FCC_0.33', 'FCC_0.67', 'FCC_1.00']].copy()
no_ss['Blocks'] = 11 - no_ss['Final Stage']
no_ss['Grouping'] = no_ss.reset_index()['type'].apply(lambda x: '_'.join(x.split('_')[1:]).replace("Historic", "H")).values
no_ss['Early Stopping'] = no_ss.reset_index()['type'].str.contains("Early").apply(lambda x: 'Yes' if x else 'No').values
no_ss['$/Block'] = no_ss[COST] / (11 - no_ss['Final Stage'])
no_ss['Value Loss / Block'] = no_ss[EFFICIENCY] / no_ss['Blocks']

no_ss['Grouping'] = no_ss['Grouping'].apply(lambda x: '1.00 (Pop)' if x == '1.00' else x)

no_ss['Mobile Licenses'] = no_ss['Blocks']

final_df = no_ss[no_ss['Grouping'].isin(['0.67', '1.00 (Pop)'])]

pivot = final_df.reset_index().pivot_table(index=['auction', 'Grouping'],columns='Early Stopping', values='Mobile Licenses')
pivot['Diff'] = pivot['Yes'] - pivot['No']
# pivot['Diff'].po09

# for g, df in final_df.groupby('Grouping'):
#     display(df.melt(id_vars='Grouping', value_vars='Mobile Licenses'))

# for g, df in final_df.groupby(['Grouping', 'Early Stopping']):
#     print(g, df['Mobile Licenses'].mean())


# plt.figure(figsize=FIGSIZE)
# for g, df in final_df.groupby(['Grouping', 'Early Stopping']):
#     s = ECDF(df['Mobile Licenses'])
#     plt.plot(s.x, s.y, label=g, linestyle='--' if g[0] == '0.67' else '-', color='r' if g[1] == 'Yes' else 'b')
# plt.xlabel('Mobile Licenses')
# plt.ylabel('Fraction of Simulations')
# plt.legend()
# g = sns.catplot(x='Grouping', y='Mobile Licenses', hue='Early Stopping', data=final_df, height=15, jitter=True)

# special_save_fig(g, os.path.join(FIGDIR,'ES_Blocks.pdf'))

# g = sns.catplot(x="Grouping", y="$/Block", hue="Early Stopping", data=es_df, height=15, kind="bar")
# special_save_fig(g, os.path.join(FIGDIR,'ES_DollarBlock.pdf'))
# g = sns.catplot(x="Grouping", y="Value Loss / Block", hue="Early Stopping", data=es_df, height=15, kind="bar")
# special_save_fig(g, os.path.join(FIGDIR,'ValueLossBlock.pdf'))

In [ ]:

# es_df.loc['Early_0.67'].groupby(['model', 'UHF_Only'])[COST].mean() / es_df.loc['SingleStage_0.67'].groupby(['model', 'UHF_Only'])[COST].mean()
# es_df.loc['Early_0.67'].groupby(['model', 'UHF_Only'])[EFFICIENCY].mean() / es_df.loc['SingleStage_0.67'].groupby(['model', 'UHF_Only'])[EFFICIENCY].mean()

# 1/(es_df.loc['Early_1.00'].groupby(['model', 'UHF_Only'])[COST].mean() / es_df.loc['SingleStage_1.00'].groupby(['model', 'UHF_Only'])[COST].mean())
# 1/(es_df.loc['Early_1.00'].groupby(['model', 'UHF_Only'])[EFFICIENCY].mean() / es_df.loc['SingleStage_1.00'].groupby(['model', 'UHF_Only'])[EFFICIENCY].mean())

es_df.loc[['Early_0.67', 'Early_1.00', 'FCC_0.67', 'FCC_1.00', 'SingleStage_0.67', 'SingleStage_1.00']]['cputime'].sum()


In [ ]:
fc_df = parse_experiment(make_folders('/global/scratch/satfc/new_experiments/v6/Afeasibility_checker'), skip_failures=True)
fc_df = fc_df.query(f"type != '{SATFC_PLUS_CPLEX}'")

In [ ]:
(fc_df.loc[GNOVELTY].groupby(['model', 'UHF_Only'])[COST].mean() / fc_df.loc['FCC'].groupby(['model', 'UHF_Only'])[COST].mean()).max()



In [ ]:
# fc_df = fc_df.drop(['FCC', 'FCC2', 'FCC_PLUS_CPLEX_V2'], level=0)
# fc_df = fc_df.rename(index={'FCC3': 'FCC'})
fc_df.reset_index()['type'].value_counts()

In [ ]:
fc_df['cputime'].sum()

In [ ]:
SATFC = 'FCC'
GUROBI = 'GUROBI'
PICOSAT = 'PICOSAT'
GREEDY = 'GREEDY'
GNOVELTY = 'GNOVELTY'
CPLEX = 'CPLEX'
SATFC_PLUS_CPLEX = 'FCC_PLUS_CPLEX_V2'

d = {
    SATFC: {
        'color': 'red',
        'label': 'SATFC',
#         'zorder': 1000
    },
    GUROBI: {
        'color': 'blue',
        'label': 'Gurobi'
    },
    PICOSAT: {
        'color': 'orange',
        'label': 'PicoSAT',
    },
    GREEDY: {
        'color': 'green',
        'label': 'Greedy'
    },
    GNOVELTY: {
        'color': 'black',
        'label': 'gnovelty+pcl'
    },
    CPLEX: {
        'color': 'purple',
        'label': 'CPLEX'
    },
    SATFC_PLUS_CPLEX: {
        'color': 'cyan',
        'label': 'SATFC+'
    }
}

dual_standard_analysis(d, 'feasibilitychecker', fc_df)

# normalized = False
# standard_analysis(d, 'feasibilitychecker123', fc_df.query(f"type != '{SATFC_PLUS_CPLEX}' and UHF_Only"))
# normalized = True

In [ ]:
## VHF
bd = '/global/scratch/satfc/new_experiments/v6/vhf'
folders = glob.glob(f'{bd}*/FCC/*')
vhf_df = parse_experiment(folders, skip_failures=True)

display(vhf_df.groupby(['model', 'type', 'UHF_Only']).count())

vhf_df = vhf_df.reset_index().drop('type', axis=1)
vhf_df['type'] = vhf_df.apply(lambda x: 'No VHF' if x['UHF_Only'] else 'FCC', axis=1)
vhf_df['UHF_Only'] = True
vhf_df = vhf_df.set_index(['type', 'auction'])


In [ ]:
print(vhf_df['cputime'].sum())

In [ ]:
VHF = 'FCC'
NO_VHF = 'No VHF'
TYPES = [VHF, NO_VHF]
FRAMES = make_frames(vhf_df, types=TYPES)


d = standard_pallet(TYPES)
d[VHF]['label'] = 'With VHF'
d[NO_VHF]['label'] = 'Without VHF'


dual_standard_analysis(d, 'vhf', vhf_df, types=TYPES)




In [ ]:
### Bid Processing Algorithm
bp_df = parse_experiment(make_folders('/global/scratch/satfc/new_experiments/v6/bidprocessing29'), skip_failures=True)

In [ ]:
FCC = 'FCC'
F2F = 'F2F'
F2F30 = 'F2F30'
TYPES = [FCC, F2F]
print(f"CPU time for f2f: {bp_df.loc[TYPES]['cputime'].sum()}")
FRAMES = make_frames(bp_df, types=TYPES)

d = standard_pallet(TYPES)

d[F2F]['label'] = 'First to Finish'
d[FCC]['label'] = r'Incentive Auction'
# d[F2F30]['label'] = 'First to Finish (0.5hr)'

dual_standard_analysis(d, 'bid_processing', bp_df.query('type != "F2F30"'), types=TYPES)

In [ ]:
bp_df.reset_index().groupby(['type', 'model'])['walltime'].mean()